In [1]:
import os
import csv
import datetime

import numpy as np
import pandas as pd

from lxml import html

In [2]:
def text_or_none(tag):
    if len(tag):
        if tag[0] is not None:
            return tag[0].text.rstrip().lstrip()
    else:
        return None
    
def parse_position(page):

    tree = html.fromstring(page)

    product_title = text_or_none(tree.xpath('//h1[@class="title"]'))
    product_subtitle = text_or_none(tree.xpath('//h2[@class="subtitle"]'))

    for tag in tree.xpath('//li'):
        data = tag.xpath('div[@class="data"]')
        title = tag.xpath('div[@class="title"]')
        if not title:
            continue
        else:
            title = title[0]
            av_storage = \
                text_or_none(title.xpath('div[@class="av_storage"]/span'))
            text_muted_subregion = \
                text_or_none(title.xpath('div[@class="text-muted subregion"]'))
            title_full_text = title.text_content().replace('\n', '').rstrip().lstrip()
        if not data:
            continue
        else:
            data = data[0]
            weight = \
                text_or_none(data.xpath('div[@class="av_nal"]/span')).replace('г', '')
            price_btc = \
                text_or_none(data.xpath('div[@class="av_price"]/span'))
            price = \
                text_or_none(data.xpath('div[@class="av_price"]/b'))\
                    .replace('руб', '')
        product = (product_title,
                   product_subtitle,
                   title_full_text,
                   text_muted_subregion,
                   av_storage,
                   weight,
                   price,
                   price_btc)
        return product
    
def parse_comments(tree_comment):
    page_comments = list()
    comments_xpath = '//li/div[@class="comment"]'
    deals_count_xpath = 'a/div[@class="review-user-data text-muted"]'
    user_name_xpath = 'div[@class="comment_cont"]/div[@class="comment_top"]/a/b'
    comment_date_xpath = 'div[@class="comment_cont"]/div[@class="comment_top"]\
        /span[@class="date text-muted"]'
    comment_product_xpath = 'div[@class="comment_cont"]/div[@class="comment_top"]\
        /div[@class="pull-right"]/a'
    comment_text_xpath = 'div[@class="comment_cont"]/div[@class="text clearfix"]'
    comment_rate_xpath = 'div[@class="comment_cont"]/div[@class="comment_foot"]\
        /div[@class="rating_star"]'
    comment_city_xpath = 'div[@class="comment_cont"]/div[@class="comment_foot"]\
        /i[@class="text-muted pull-right"]'
    user_link_xpath = 'a[@class="cover avatar"]'
    comments = tree_comment.xpath(comments_xpath)

    for tagzor in comments:
        try:
            comment_parsed = dict()
            comment_parsed['deals_count'] = \
                text_or_none(tagzor.xpath(deals_count_xpath))
            if comment_parsed['deals_count']:
                comment_parsed['deals_count'] = \
                    comment_parsed['deals_count'].split(': ')[1]
            comment_parsed['user_name'] = \
                text_or_none(tagzor.xpath(user_name_xpath))
            comment_parsed['comment_date'] = \
                text_or_none(tagzor.xpath(comment_date_xpath))
            comment_parsed['comment_text'] = \
                text_or_none(tagzor.xpath(comment_text_xpath))
            comment_parsed['comment_city'] = \
                text_or_none(tagzor.xpath(comment_city_xpath))
            comment_parsed['user_link'] = \
                tagzor.xpath(user_link_xpath)[0].get('href')
            comment_parsed['comment_product_link'] = \
                tagzor.xpath(comment_product_xpath)

            if comment_parsed['comment_product_link']:
                comment_parsed['comment_product_link'] = \
                    comment_parsed['comment_product_link'][0].get('href')
                comment_parsed['user_type'] = 'buyer'
            else:
                comment_parsed['comment_product_link'] = None
                comment_parsed['user_type'] = 'seller'
            comment_parsed['comment_product_text'] = \
                text_or_none(tagzor.xpath(comment_product_xpath))
            comment_parsed['comment_rate'] = \
                tagzor.xpath(comment_rate_xpath)

            if comment_parsed['comment_rate']:
                comment_parsed['comment_rate'] = \
                    comment_parsed['comment_rate'][0].get('href')
                comment_parsed['user_type'] = 'buyer'
            else:
                comment_parsed['comment_rate'] = None
                comment_parsed['user_type'] = 'seller'

            yield comment_parsed
        except:
            continue

In [3]:
def parse_star(tag):
    tag_href = tag.get('href')
    tag_text = tag.text
    if '%' in tag_text:
        return None
    rating = tag_href.split('rate=')[1]
    counter = tag_text.split(' ')[0]
    return (rating, counter)

def parse_market(tree_page):
    market_parsed = dict()
    shop_deals_count_xpath = '//div[@class="header_shop__info"]/ul[@class="text-muted"]/li/a'
    shop_header_xpath = '//div[@class="header_shop__info"]/h1[@class="title over"]'
    review_rate_xpath = '//div[@class="col-md-3 col-sm-4"]'
    review_rate_mark_xpath = \
        '//div[@class="col-md-3 col-sm-4"]/div[@class="review-rate-filter"]/small/a'
    
    for rate_mark in tree_page.xpath(review_rate_mark_xpath):
        rate = parse_star(rate_mark)
        if rate:
            market_parsed[rate[0]] = rate[1]
    market_parsed['shop_header'] = \
        text_or_none(tree_page.xpath(shop_header_xpath))
#     market_parsed['shop_link'] = \
#         tree_page.xpath(shop_deals_count_xpath)[0].get('href')
    market_parsed['shop_deals_count'] = \
        text_or_none(tree_page.xpath(shop_deals_count_xpath))
    return market_parsed

In [72]:
product_categories = [
    'Хакинг',
    'Цифровые товары',
    'Семена',
    'Аптека',
    'Эйфоретики',
    'Бошка',
    'Кокаин',
    'Гашиш',
    'Базы',
    'SSH, VPN',
    'Альфа-ПВП',
    'Мефедрон',
    'Сканы',
    'Приборы и оборудование',
    'Обнал BTC',
    'Масло/Экстракты', 
    'Амфетамин', 
    'MDMA', 
    'Банковские карты',
    'Документы', 
    'Кошельки', 
    'Разное', 
    'Пробив', 
    'Экстази', 
    'LSD',
    'Автодокументы', 
    'Cтимуляторы', 
    'Химические реактивы', 
    'Марихуана',
    'Дизайн и графика', 
    'Метадон', 
    'Триптамины', 
    'Другое',
    'Диссоциативы', 
    'Наружная реклама', 
    'Анаболики/Стероиды',
    'Химические реактивы/Конструкторы', 
    'Конструкторы', 
    'План',
    'Метамфетамин', 
    'Героин', 
    'Грибы', 
    'SIM-карты', 
    'Карты, SIM',
    'CannaFood', 
    'Реагент', 
    'DMT', 
    'Удостоверения', 
    'Бутираты', 
    '2С-*',
    'Фальшивые деньги', 
    'NBOME', 
    'DO*', 
    'Твердый Микс', 
    'Каннабиноиды',
    '1-фенил-2-нитропропен', 
    'Мескалин', 
    'Россыпь/Микс', 
    'MDA', 
    'MDPV',
    'Энтеогены', 
    'Основы для смесей', 
    'Опиаты', 
    'Катиноны',
    'Психоделики', 
    'Эфедрон', 
    'Винт', 
    'Майнинг криптовалют'
]

drugs = [
        'Аптека', 'Эйфоретики','Бошка', 'Кокаин', 'Гашиш', 'Альфа-ПВП',
        'Мефедрон', 'Масло/Экстракты', 'Амфетамин', 'MDMA', 'Экстази', 'LSD',
        'Cтимуляторы', 'Марихуана','Метадон', 'Триптамины',
        'Диссоциативы', 'Анаболики/Стероиды', 'План',
        'Метамфетамин', 'Героин', 'Грибы', 'Мука', 'Кристалл',
        'CannaFood', 'Реагент', 'DMT', 'Бутираты', '2С-*',
        'NBOME', 'DO*', 'Твердый Микс', 'Каннабиноиды',
        '1-фенил-2-нитропропен', 'Мескалин', 'Россыпь/Микс', 'MDA', 'MDPV',
        'Энтеогены', 'Опиаты', 'Катиноны',
        'Психоделики', 'Эфедрон', 'Винт'
        ]

jobs = [
    'Партнёрство и Франшиза',
    'Работа',
    'Курьер/Перевозчик',
    'Другая работа',
    'Трафаретчик',
    'Кладмен',
    'Наружная реклама',
    'Склад',
    'Дизайн и графика',
    'Промоутер'
]

production = [
    'Реагент',
    'Химические реактивы/Конструкторы', 
    'Конструкторы',
    'Приборы и оборудование',
    'Химические реактивы',
    'Семена'
]

operations = [
    'Обнал BTC',
    'Дизайн и графика',
    'Наружная реклама',
    'Майнинг криптовалют',
    'Банковские карты',
    'SIM-карты', 
    'Карты, SIM',
    'Фальшивые деньги',
]

In [17]:
directory = 'catalog/page/product'

all_products = list()
errors = 0
fieldnames = ['Название позиции',
              'Тип позиции',
              'Полное описание',
              'Уточненная локация',
              'Тип закладки',
              'Масса',
              'Стоимость',
              'price_btc']

for filename in os.listdir(directory):
    filepath = directory + '/' + filename
    try:
        with open(filepath) as f:
            read_data = f.read()
            product = parse_position(read_data)
    except:
        errors += 1
        continue
    if product:
        product_parsed = dict(zip(fieldnames, product))
        product_parsed['id'] = filename
        all_products.append(product_parsed)

In [135]:
# можно засунуть внутрь цикла выше
# все вытащить, потом обрабатывать — надежнее

for product in all_products:
    product['title_full_text_replaced'] = product['Полное описание']\
        .replace('Прикоп', '')\
        .replace('Тайник', '')\
        .replace('Магнит', '')\
        .replace(', ', '')\
        .replace(',','')\
        .replace('\
        ','')\
        .rstrip()\
        .lstrip()\
        .split(':')[0]
    product['Стоимость'] = product['Стоимость']\
                            .replace(' ', '')
    if 'шт' in product['Масса']:
        product['Мера'] = 'штуки'
        product['Масса'] = product['Масса'].replace('\xa0шт', '').replace('\xa0', '')
    elif 'мл' in product['Масса']:
        product['Мера'] = 'мл'
        product['Масса'] = product['Масса'].replace('мл', '').replace('\xa0', '')
    elif 'м' in product['Масса']:
        product['Мера'] = 'м'
        product['Масса'] = product['Масса'].replace('м', '').replace('\xa0', '')
    else:
        product['Мера'] = 'граммы'
        product['Масса'] = product['Масса'].replace('\xa0', '')
    if '$' in product['Стоимость']:
        product['Валюта'] = 'доллар'
        product['Стоимость'] = product['Стоимость'].replace('$', '')
    else:
        product['Валюта'] = 'рубль'
    product['Тип позиции'] = product['Тип позиции'].replace('Кристалл', 'Мефедрон')
    product['Тип позиции'] = product['Тип позиции'].replace('Бошка', 'Марихуана')
    if 'PVP' not in product['Название позиции']:
        product['Тип позиции'] = product['Тип позиции'].replace('Мука', 'Мефедрон')
    elif 'PVP' in product['Название позиции']:
        product['Тип позиции'] = 'Альфа-ПВП'
    if '(' in product['title_full_text_replaced']:
        product['title_full_text_replaced'] = \
            product['title_full_text_replaced'].split(' (')[0]
df = pd.DataFrame(all_products)
df.drop('price_btc', axis=1, inplace=True)
df = df.astype({'Стоимость': 'int64', 'Масса': 'float'})
df['Город'] = df['title_full_text_replaced']
df.drop('title_full_text_replaced', inplace=True, axis=1)
df.drop(2606, inplace=True)

cities_df = pd.read_csv('./cities.csv')
cities_df.drop(['Индекс', 'Тип города', 'Тип региона', 'Тип района', 'Район',
                'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
                'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
                'Код ИФНС'], inplace=True, axis=1)

In [145]:
df_drug = df[df['Тип позиции'].isin(drugs)]
# df_drug = df_merged[df_merged['Тип позиции'].isin(drugs)]
group_by_type = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby('Тип позиции')\
        .sum()

group_by_type_count = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby('Тип позиции')\
        .count()

group_by_type['Масса кг'] = group_by_type['Масса']/1000
group_by_type.drop('Масса', axis=1, inplace=True)

group_by_type['Количество предложений в день'] = group_by_type_count['id']

group_by_type['Месячная стоимость'] = group_by_type['Стоимость']*26

group_by_type['Месячная стоимость в панамерах'] = \
    group_by_type['Месячная стоимость']/15000000
group_by_type['Месячная стоимость в айфонах'] = \
    group_by_type['Месячная стоимость']/80000

group_by_type.sort_values('Стоимость', ascending=False)
group_by_type.to_csv('group_by_type.csv')

In [146]:
group_by_city = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby(['Город'])\
        .sum()

drug_by_city_count = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby(['Город'])\
        .count()

group_by_city['Масса кг'] = group_by_city['Масса']/1000
group_by_city.drop('Масса', axis=1, inplace=True)

group_by_city['Количество предложений в день'] = drug_by_city_count['id']

group_by_city['Месячная стоимость'] = group_by_city['Стоимость']*26

group_by_city['Месячная стоимость в панамерах'] = \
    group_by_city['Месячная стоимость']/15000000
group_by_city['Месячная стоимость в айфонах'] = \
    group_by_city['Месячная стоимость']/80000

group_by_city.sort_values('Стоимость', ascending=False)
group_by_city.to_csv('group_by_city.csv')

In [156]:
group_by_city_type = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby(['Город','Тип позиции'])\
        .sum()

group_by_city_type_count = \
    df_drug\
        [df_drug['Мера'] == 'граммы']\
        [df_drug['Масса'] < 1000]\
        .groupby(['Город','Тип позиции'])\
        .count()

group_by_city_type['Масса кг'] = group_by_city_type['Масса']/1000
group_by_city_type.drop('Масса', axis=1, inplace=True)

group_by_city_type['Количество предложений в день'] = group_by_city_type_count['id']

group_by_city_type.to_csv('group_by_city_type.csv')

In [ ]:
all_comments = list()
files_done = list()
i = 0
comments_market_dir = \
    'catalog/page/expiriences/market'
for market in os.listdir(comments_market_dir):
    market_products = comments_market_dir + '/' + market
    for product in os.listdir(market_products):
        product_pages = market_products + '/' + product
        pid = product.split('product_id=')[1]
        for page in os.listdir(product_pages):
            filepath = product_pages + '/' + page
            if i%10000 == 0:
                print(i)
            with open(filepath) as f:
                try:
                    read_data = f.read()
                    tree_comment = html.fromstring(read_data)
                except:
                    continue
            for comment in parse_comments(tree_comment):
                comment['market'] = market
                comment['pid'] = pid
                comment['page'] = page
                all_comments.append(comment)
            files_done.append(filepath)
            i += 1

In [ ]:
# можно засунуть внутрь цикла выше
# все вытащить, потом обрабатывать — надежнее

for comment in all_comments:
    date, time = comment['comment_date'].split(" в ")
    if 'сегодня' in date:
        comment['date'] = '05.08.2019'
        comment['time'] = time
        continue
    if 'вчера' in date:
        comment['date'] = '04.08.2019'
        comment['time'] = time
        continue
    if '2018' not in date and '2017' not in date:
        date = date + ' 2019'
    date = date\
        .replace(' Января ','.01.')\
        .replace(' Февраля ','.02.')\
        .replace(' Марта ','.03.')\
        .replace(' Апреля ','.04.')\
        .replace(' Мая ','.05.')\
        .replace(' Июня ','.06.')\
        .replace(' Июля ','.07.')\
        .replace(' Августа ','.08.')\
        .replace(' Сентября ','.09.')\
        .replace(' Октября ','.10.')\
        .replace(' Ноября ','.11.')\
        .replace(' Декабря ','.12.')
    comment['date'] = date
    comment['time'] = time

dfc = pd.DataFrame(all_comments)
dfc.to_csv('comments_fixed.csv')

In [ ]:
dfc['pub_date'] = pd.to_datetime(dfc['date'])
dfc['date'] = dfc['pub_date']

dfc.drop('pub_date', axis=1, inplace=True)

dfct = dfc.set_index('date')

df_month = \
    dfct[dfct['user_type']== 'buyer'][dfct['pub_date'] < datetime.date(2019, 9, 1)]\
    .resample('M').count()
df_month[:'2019-8-1 00:00:00']['user_name'].to_csv('reviews.csv')

In [ ]:
all_markets = list()
markets_dir = 'catalog/page/markets'
for market in os.listdir(markets_dir):
    market_path = markets_dir + '/' + market
    with open(market_path) as f:
        try:
            read_data = f.read()
            tree = html.fromstring(read_data)
        except:
            continue
    market_parsed = parse_market(tree)
    all_markets.append(market_parsed)

In [ ]:
for market in all_markets:
    if market['shop_deals_count']:
        if '+' in market['shop_deals_count']:
            market['shop_deals_count_true'] = market['shop_deals_count'].split('+')[0]
        elif 'сдел' in market['shop_deals_count']:
            market['shop_deals_count_true'] = \
                market['shop_deals_count']\
                    .replace('сделок', '')\
                    .replace('сделки', '')\
                    .replace('сделка', '')
    else:
        market['shop_deals_count_true'] = 0
dfm = pd.DataFrame(all_markets)
dfm.drop('shop_deals_count', axis=1, inplace=True)
dfm.drop(966, inplace=True)

dfm['deals'] = dfm['shop_deals_count_true'].astype('int64')

dfm1 = dfm.dropna(subset=['0']).astype({
    '10':'int64',
    '8':'int64',
    '6':'int64',
    '4':'int64',
    '2':'int64',
    '0':'int64',
    'shop_deals_count_true':'int64'
})

dfm1['summary_comments'] = \
    dfm1['10'] + dfm1['8'] + dfm1['6'] + dfm1['4'] + dfm1['2'] + dfm1['0']
dfm1['commented_share'] = \
    dfm1['summary_comments']/dfm1['shop_deals_count_true'] * 100

dfm1 = dfm1[dfm1['commented_share'] < 100]
dfm1.to_csv('./markets_full.csv')

In [180]:
users_dir = 'user'

all_users = list()

for filename in os.listdir(users_dir):
    filepath = users_dir + '/' + filename
    try:
        with open(filepath) as f:
            read_data = f.read()
        tree = html.fromstring(read_data)
        user = dict()

        user['username'] = \
            text_or_none(tree.xpath('//div[@class="title h1 over"]'))
        user['regdate'] = \
            text_or_none(tree.xpath('//div[@class="col-xs-12 col-md-5"]/div/b'))
        all_users.append(user)
    except:
        continue

In [182]:
udf = pd.DataFrame(all_users)
udf['date'] = pd.to_datetime(udf['regdate'])
udf.drop('regdate', axis=1, inplace=True)
udf = udf.set_index('date')

udf_month = udf.resample('M').count()
udf_month[:'2019-8-1 00:00:00'].to_csv('new_users_by_month.csv')

In [ ]:
directory = 'catalog/page/product'

all_products = list()
errors = 0
fieldnames = ['Название позиции',
              'Тип позиции',
              'Полное описание',
              'Уточненная локация',
              'Тип закладки',
              'Масса',
              'Стоимость',
              'price_btc']

for filename in os.listdir(directory):
    filepath = directory + '/' + filename
    try:
        with open(filepath) as f:
            read_data = f.read()
            product = parse_position(read_data)
    except:
        errors += 1
        continue
    if product:
        product_parsed = dict(zip(fieldnames, product))
        product_parsed['id'] = filename
        all_products.append(product_parsed)